In [1]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [2]:
local_zip = "/springboard/Unit 24/mask.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall("/springboard/Unit 24/")
zip_ref.close()

In [3]:
base_dic = "E:/springboard/Unit 24/mask"
train_dic = os.path.join(base_dic, "train")
validation_dic = os.path.join(base_dic, "validation")

train_mask_dic = os.path.join(train_dic, "with_mask")

train_non_mask_dic = os.path.join(train_dic, "without_mask")

validation_mask_dic = os.path.join(validation_dic, "with_mask")

validation_non_mask_dic = os.path.join(validation_dic, "without_mask")

In [59]:
# Set up matplotlib fig, and size it to fit 4x4 pics
import matplotlib.image as mpimg
nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index = 100
train_mask_fnames = os.listdir( train_mask_dic )
train_non_mask_fnames = os.listdir( train_non_mask_dic )


next_mask_pix = [os.path.join(train_mask_dic, fname) 
                for fname in train_mask_dic[ pic_index-8:pic_index] 
               ]

next_non_mask_pix = [os.path.join(train_non_mask_fnames, fname) 
                for fname in train_non_mask_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_mask_pix + next_non_mask_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

<Figure size 1152x1152 with 0 Axes>

In [7]:
# rescale all picture size

train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [8]:
train_generator = train_datagen.flow_from_directory(train_dic, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( validation_dic,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 122 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


In [9]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

C:\Users\lv164\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [12]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 30s 278ms/step - loss: 1.8096 - acc: 0.5656 - val_loss: 0.5555 - val_acc: 0.6572


# Inception

In [13]:
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [14]:
train_generator = train_datagen.flow_from_directory(train_dic, batch_size = 20, class_mode = 'binary', target_size = (150, 150))
validation_generator = test_datagen.flow_from_directory(validation_dic, batch_size = 20, class_mode = 'binary', target_size = (150, 150))

Found 122 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


In [15]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

In [16]:
for layer in base_model.layers:
    layer.trainable = False

In [17]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [18]:
inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


<ipython-input-18-8d91753fef64>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


Epoch 1/10
100/100 [==============================] - 11s 77ms/step - loss: 6.4209 - acc: 0.6885 - val_loss: 2.1950 - val_acc: 0.7264


# ResNet 50

In [19]:
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dic, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( validation_dic, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 122 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


In [20]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [21]:
for layer in base_model.layers:
    layer.trainable = False

In [22]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD 
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

In [23]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])


C:\Users\lv164\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [24]:
resnet_history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 33s 271ms/step - loss: 2.6425 - acc: 0.5656 - val_loss: 1.0777 - val_acc: 0.5000


# EfficientNet

In [43]:
from tensorflow.keras.applications import EfficientNetB0 

In [55]:
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dic, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( validation_dic, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 122 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


In [56]:
base_model = EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [57]:
for layer in base_model.layers:
    layer.trainable = False

In [58]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(input = base_model.input, output = predictions)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.